# Project with Solution (Section 7)

## Project Specs

This is where we left off in the previous section project:

In [1]:
from datetime import date
from enum import Enum
from uuid import uuid4
from pydantic import BaseModel, ConfigDict, Field, field_serializer
from pydantic.alias_generators import to_camel
from pydantic import UUID4


class AutomobileType(Enum):
    sedan = "Sedan"
    coupe = "Coupe"
    convertible = "Convertible"
    suv = "SUV"
    truck = "Truck"


class Automobile(BaseModel):
    model_config = ConfigDict(
        extra="forbid",
        str_strip_whitespace=True,
        validate_default=True,
        validate_assignment=True,
        alias_generator=to_camel,
    )

    id_: UUID4 | None = Field(alias="id", default_factory=uuid4) 
    manufacturer: str
    series_name: str
    type_: AutomobileType = Field(alias="type")
    is_electric: bool = False
    manufactured_date: date = Field(validation_alias="completionDate", ge=date(1980, 1, 1))
    base_msrp_usd: float = Field(
        validation_alias="msrpUSD", 
        serialization_alias="baseMSRPUSD"
    )
    vin: str
    number_of_doors: int = Field(
        default=4, 
        validation_alias="doors",
        ge=2,
        le=4,
        multiple_of=2,
    )
    registration_country: str | None = None
    license_plate: str | None = None

    @field_serializer("manufactured_date", when_used="json-unless-none")
    def serialize_date(self, value: date) -> str:
        return value.strftime("%Y/%m/%d")

Create an annotated type, named `BoundedString` to define a string that has a minimum of 2 characters, and no more than 50 characters.

Modify your `Automobile` model to use this annotated type for the following fields:
- `manufacturer`
- `series_name`
- `vin`
- `registration_country`
- `license_plate`

Create an annotated type, called `BoundedList` that uses a type variable to define a list of elements with a minimum of `1` element and a maximum of `5` elements.

Using this annotated type, add a new field to the model as follows:
- field name should be `top_features`
- place it just before the `vin` field
- it should both deserialize from and serialize to `topFeatures`
- if should be a bounded list of strings, which themselves shoudl be bounded to a minimum of `2` chars, and no more than `50`. (Hint: use the `BoundedString` type you create as the type when you define the field type in your model with `BoundedList`)
- make it optional, with a default of `None`

Use this data to test your model:

In [2]:
from uuid import UUID

data = {
    "id": "c4e60f4a-3c7f-4da5-9b3f-07aee50b23e7",
    "manufacturer": "BMW",
    "seriesName": "M4 Competition xDrive",
    "type": "Convertible",
    "isElectric": False,
    "completionDate": "2023-01-01",
    "msrpUSD": 93_300,
    "topFeatures": ["6 cylinders", "all-wheel drive", "convertible"],
    "vin": "1234567890",
    "doors": 2,
    "registrationCountry": "France",
    "licensePlate": "AAA-BBB"
}

expected_serialized_by_alias = {
    'id': UUID('c4e60f4a-3c7f-4da5-9b3f-07aee50b23e7'),
    'manufacturer': 'BMW',
    'seriesName': 'M4 Competition xDrive',
    'type': AutomobileType.convertible,
    'isElectric': False,
    'manufacturedDate': date(2023, 1, 1),
    'baseMSRPUSD': 93300.0,
    'topFeatures': ['6 cylinders', 'all-wheel drive', 'convertible'],
    'vin': '1234567890',
    'numberOfDoors': 2,
    'registrationCountry': 'France',
    'licensePlate': 'AAA-BBB'
}

You should also test that you get validation errors if you attempt to deserialize data that does not conform to the constraints. 

> Suggestion: create simple single field models to just test your Annotated types and make sure they work as expected. Once you have done that, go ahead and use them in your main model.

## Solution

In [3]:
from typing import Annotated, TypeVar
from pydantic import Field

BoundedString = Annotated[str, Field(min_length=2, max_length=50)]

T = TypeVar('T')

BoundedList = Annotated[list[T], Field(min_length=1, max_length=5)]

Let's test these out before we use them in our model:

In [4]:
class Test(BaseModel):
    field1: BoundedString

In [5]:
Test(field1="abc")

Test(field1='abc')

In [6]:
from pydantic import ValidationError

In [7]:
try:
    Test(field1="a")
except ValidationError as ex:
    print(ex)

1 validation error for Test
field1
  String should have at least 2 characters [type=string_too_short, input_value='a', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/string_too_short


In [8]:
try:
    Test(field1="a" * 51)
except ValidationError as ex:
    print(ex)

1 validation error for Test
field1
  String should have at most 50 characters [type=string_too_long, input_value='aaaaaaaaaaaaaaaaaaaaaaaa...aaaaaaaaaaaaaaaaaaaaaaa', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/string_too_long


In [9]:
class Test(BaseModel):
    my_list: BoundedList[int]

In [10]:
Test(my_list=[1, 2, 3])

Test(my_list=[1, 2, 3])

In [11]:
try:
    Test(my_list=[])
except ValidationError as ex:
    print(ex)

1 validation error for Test
my_list
  List should have at least 1 item after validation, not 0 [type=too_short, input_value=[], input_type=list]
    For further information visit https://errors.pydantic.dev/2.5/v/too_short


In [12]:
try:
    Test(my_list=[1, 2, 3, 4, 5, 6])
except ValidationError as ex:
    print(ex)

1 validation error for Test
my_list
  List should have at most 5 items after validation, not 6 [type=too_long, input_value=[1, 2, 3, 4, 5, 6], input_type=list]
    For further information visit https://errors.pydantic.dev/2.5/v/too_long


And now we can combine these two to make a bounded list of bounded strings:

In [13]:
class Test(BaseModel):
    my_list: BoundedList[BoundedString]

In [14]:
Test(my_list=['aa', 'bb', 'cc'])

Test(my_list=['aa', 'bb', 'cc'])

In [15]:
try:
    Test(my_list=[])
except ValidationError as ex:
    print(ex)

1 validation error for Test
my_list
  List should have at least 1 item after validation, not 0 [type=too_short, input_value=[], input_type=list]
    For further information visit https://errors.pydantic.dev/2.5/v/too_short


In [16]:
try:
    Test(my_list=['a', 'bb', 'cc'])
except ValidationError as ex:
    print(ex)

1 validation error for Test
my_list.0
  String should have at least 2 characters [type=string_too_short, input_value='a', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/string_too_short


In [17]:
try:
    Test(my_list=['a' * 51, 'bb', 'cc'])
except ValidationError as ex:
    print(ex)

1 validation error for Test
my_list.0
  String should have at most 50 characters [type=string_too_long, input_value='aaaaaaaaaaaaaaaaaaaaaaaa...aaaaaaaaaaaaaaaaaaaaaaa', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/string_too_long


Ok, so those seem to work, and we can now use them in our model:

In [18]:
class Automobile(BaseModel):
    model_config = ConfigDict(
        extra="forbid",
        str_strip_whitespace=True,
        validate_default=True,
        validate_assignment=True,
        alias_generator=to_camel,
    )

    id_: UUID4 | None = Field(alias="id", default_factory=uuid4) 
    manufacturer: BoundedString
    series_name: BoundedString
    type_: AutomobileType = Field(alias="type")
    is_electric: bool = False
    manufactured_date: date = Field(validation_alias="completionDate", ge=date(1980, 1, 1))
    base_msrp_usd: float = Field(
        validation_alias="msrpUSD", 
        serialization_alias="baseMSRPUSD"
    )
    top_features: BoundedList[BoundedString] | None = None
    vin: BoundedString
    number_of_doors: int = Field(
        default=4, 
        validation_alias="doors",
        ge=2,
        le=4,
        multiple_of=2,
    )
    registration_country: BoundedString | None = None
    license_plate: BoundedString | None = None

    @field_serializer("manufactured_date", when_used="json-unless-none")
    def serialize_date(self, value: date) -> str:
        return value.strftime("%Y/%m/%d")

In [19]:
data = {
    "id": "c4e60f4a-3c7f-4da5-9b3f-07aee50b23e7",
    "manufacturer": "BMW",
    "seriesName": "M4 Competition xDrive",
    "type": "Convertible",
    "isElectric": False,
    "completionDate": "2023-01-01",
    "msrpUSD": 93_300,
    "topFeatures": ["6 cylinders", "all-wheel drive", "convertible"],
    "vin": "1234567890",
    "doors": 2,
    "registrationCountry": "France",
    "licensePlate": "AAA-BBB"
}

expected_serialized_by_alias = {
    'id': UUID('c4e60f4a-3c7f-4da5-9b3f-07aee50b23e7'),
    'manufacturer': 'BMW',
    'seriesName': 'M4 Competition xDrive',
    'type': AutomobileType.convertible,
    'isElectric': False,
    'manufacturedDate': date(2023, 1, 1),
    'baseMSRPUSD': 93300.0,
    'topFeatures': ['6 cylinders', 'all-wheel drive', 'convertible'],
    'vin': '1234567890',
    'numberOfDoors': 2,
    'registrationCountry': 'France',
    'licensePlate': 'AAA-BBB'
}

In [20]:
car = Automobile.model_validate(data)
car

Automobile(id_=UUID('c4e60f4a-3c7f-4da5-9b3f-07aee50b23e7'), manufacturer='BMW', series_name='M4 Competition xDrive', type_=<AutomobileType.convertible: 'Convertible'>, is_electric=False, manufactured_date=datetime.date(2023, 1, 1), base_msrp_usd=93300.0, top_features=['6 cylinders', 'all-wheel drive', 'convertible'], vin='1234567890', number_of_doors=2, registration_country='France', license_plate='AAA-BBB')

In [21]:
assert car.model_dump(by_alias=True) == expected_serialized_by_alias